구글 드라이브 연결

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


ㅇㄹ

In [5]:
# 라이브러리 import
import pandas as pd

# 7번 테이블 연평균 해수면 온도 이상값
sst_path = "/content/drive/MyDrive/sea-surface-temperature-anomaly.csv"

# 파일 불러오기
df_sst = pd.read_csv(sst_path)

# 컬럼 확인 (참고용)
print("[SST 파일 컬럼]")
print(df_sst.columns.tolist())
print()

# Code 컬럼 삭제
df_sst = df_sst.drop(columns=["Code"])

# 컬럼명 재정의
df_sst.columns = [
    "region_entity",
    "measurement_year",
    "sst_anomaly_celsius",
    "anomaly_lower_bound",
    "anomaly_upper_bound"
]

# 결측치 NULL 처리
df_sst = df_sst.fillna("NULL")


# INSERT SQL 생성
sst_values = []
for _, r in df_sst.iterrows():
    region = r.region_entity.replace("'", "''") if r.region_entity != "NULL" else "NULL"
    year = int(r.measurement_year) if r.measurement_year != "NULL" else "NULL"
    anomaly = r.sst_anomaly_celsius if r.sst_anomaly_celsius != "NULL" else "NULL"
    lower = r.anomaly_lower_bound if r.anomaly_lower_bound != "NULL" else "NULL"
    upper = r.anomaly_upper_bound if r.anomaly_upper_bound != "NULL" else "NULL"
    sst_values.append(f"('{region}', {year}, {anomaly}, {lower}, {upper})")

sst_sql = (
    "-- 7. Global_Annual_SST_Anomalies\n"
    "INSERT INTO global_annual_sst_anomalies (region_entity, measurement_year, sst_anomaly_celsius, anomaly_lower_bound, anomaly_upper_bound) VALUES\n"
    + ",\n".join(sst_values)
    + ";\n\n"
)

print(f"✅ SST 데이터 {len(df_sst)}건 처리 완료!")


[SST 파일 컬럼]
['Entity', 'Code', 'Year', 'Annual sea surface temperature anomalies', 'Annual sea surface temperature anomalies (lower bound)', 'Annual sea surface temperature anomalies (upper bound)']

✅ SST 데이터 528건 처리 완료!


In [10]:
# 8번 테이블 (CO2 농도)
co2_path = "/content/drive/MyDrive/co2-long-term-concentration.csv"

df_co2 = pd.read_csv(co2_path)

print("[CO2 파일 컬럼]")
print(df_co2.columns.tolist())
print()

# Code 컬럼 삭제
df_co2 = df_co2.drop(columns=["Code"])

# 컬럼 이름 통일
df_co2.columns = ["region_entity", "year", "co2_ppm"]

# 결측치 NULL 처리
df_co2 = df_co2.fillna("NULL")

co2_values = []
for _, r in df_co2.iterrows():
    year = int(r.year) if r.year != "NULL" else "NULL"
    ppm = r.co2_ppm if r.co2_ppm != "NULL" else "NULL"
    co2_values.append(f"({year}, {ppm})")

co2_sql = (
    "-- 8. CO2_Concentration\n"
    "INSERT INTO co2_concentration (year, co2_ppm) VALUES\n"
    + ",\n".join(co2_values)
    + ";\n\n"
)

print(f"✅ CO₂ 데이터 {len(df_co2)}건 처리 완료!")



[CO2 파일 컬럼]
['Entity', 'Code', 'Year', 'Annual concentration of atmospheric carbon dioxide']

✅ CO₂ 데이터 1873건 처리 완료!


SQL 파일로 만들기

In [12]:
final_sql = (
    sst_sql + co2_sql
)

output_path = "/content/drive/MyDrive/dbinsert_7_8.sql"

with open(output_path, "w", encoding="utf-8-sig") as f:
    f.write(final_sql)

print("\n✅ SQL 파일 생성 완료!")
print("📁 저장 경로:", output_path)


✅ SQL 파일 생성 완료!
📁 저장 경로: /content/drive/MyDrive/dbinsert_7_8.sql
